In [1]:
import requests
import urllib.parse

In [2]:
def get_robots_url(url):
    """
    urlをスクレイプしてよいかが記載されたrobots.txtの内容を返す
    """
    robots_url = urllib.parse.urlunparse(urllib.parse.urlparse(url)._replace(path='/robots.txt',query='',fragment=''))
    response = requests.get(robots_url)
    return response.text

In [3]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [4]:
def is_scrapable(url, user_agent):
    """
    User-Agent名 user_agentがurlをスクレイプしてよいかどうかをOpenAI APIで判定
    """
    prompt = f"""
    下記に引用した robots.txt が設置されているWebサイトから URL `{url}` を User-Agent `{user_agent}` でスクレイプしてよいかどうかをはいかいいえで回答してください。

    #robots.txtの内容:
    {get_robots_url(url)}
    """

    # APIへリクエスト
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": prompt},
        ],
        max_tokens=500,
        temperature=0.3
    )

    print(response.choices[0].message.content.strip())

https://www.kiramex.com/robots.txt

In [5]:
is_scrapable("https://www.kiramex.com/news/", "Mozilla/5.0") # はい

はい。


In [6]:
is_scrapable("https://www.kiramex.com/wp-admin/admin-ajax.php?a=1&b=2", "curl/7.36") # はい

はい。


In [7]:
is_scrapable("https://www.kiramex.com/wp-admin/admin.php", "python-requests/2.32.4") # いいえ

いいえ。


https://search.yahoo.co.jp/robots.txt

In [8]:
is_scrapable("https://search.yahoo.co.jp/search?p=1", "Googlebot-Image/0.0") # いいえ

いいえ。


In [9]:
is_scrapable("https://search.yahoo.co.jp/search?p=1", "Twitterbot/0.0") # はい


はい


In [10]:
is_scrapable("https://search.yahoo.co.jp/aichat?ml=prop%3Aai_chat%3Bfrtype%3Astaichat%3B", "Googlebot/0.0") # いいえ

いいえ


In [11]:
is_scrapable("https://search.yahoo.co.jp/aichat?ml=prop%3Aai_chat%3Bfrtype%3Astaichat%3B", "facebookexternalhit/0.0") # いいえ

はい
